# Google Serper



This notebook demonstrates how Large Language Model (LLM) responses can be significantly improved by integrating real-time web search using the Serper API. By comparing responses to the same query — one using only the LLM's internal knowledge, and the other augmented with up-to-date search results — we illustrate the value of web-enabled augmentation, especially for time-sensitive or rapidly evolving topics.



In [ ]:
%pip install --upgrade --quiet  langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


#**A basic example of Serper API for web search**

In [ ]:
import os
import pprint

os.environ["SERPER_API_KEY"] 

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

search = GoogleSerperAPIWrapper()

In [ ]:
search.run("Obama's first name?")

'Barack Hussein Obama II'

In [ ]:
import requests
API_KEY = os.environ["X-API-KEY"]
url = "https://google.serper.dev/search"
headers = {
    "X-API-KEY": API_KEY,
    "Content-Type": "application/json"
}
data = {
    "q": "Obama's first name?"
}

response = requests.post(url, headers=headers, json=data)
print(response.json())


{'message': 'Unauthorized.', 'statusCode': 403}


In [ ]:
import requests

params = {
    "engine": "google",
    "q": "Obama's first name?",
    "api_key": API_KEY
}

response = requests.get("https://serpapi.com/search", params=params)
print(response.json())


{'search_metadata': {'id': '684982b1b7942f506c6dc765', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/568a6f2ee8c5d728/684982b1b7942f506c6dc765.json', 'created_at': '2025-06-11 13:20:49 UTC', 'processed_at': '2025-06-11 13:20:49 UTC', 'google_url': 'https://www.google.com/search?q=Obama%27s+first+name%3F&oq=Obama%27s+first+name%3F&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/568a6f2ee8c5d728/684982b1b7942f506c6dc765.html', 'total_time_taken': 1.67}, 'search_parameters': {'engine': 'google', 'q': "Obama's first name?", 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': "Obama's first name?", 'total_results': 23800000, 'time_taken_displayed': 0.48, 'organic_results_state': 'Results for exact spelling'}, 'knowledge_graph': {'title': 'Barack Obama', 'type': '44th U.S. President', 'entity_type': 'people', 'kgmid': '/m/02mjmr', 'knowledge_graph_search_link': 'https://www.google.com/search?kgmid=/m/0

#**Now lets do LLM inference by augmenting search results in prompt**

In [ ]:
!pip install -qU google-generativeai


In [ ]:
import google.generativeai as genai

#**Example 1: LLM inference without search results or without using web search tool**

In this section, we test how the LLM responds without using any web search or external context. This helps establish a baseline by evaluating the model's performance based solely on its internal knowledge without any help from live data or search results.



In [ ]:
from google import genai
query = "What did Apple announce at WWDC 2025?"
prompt = f"Answer the question: \n {query}"
client = genai.Client(api_key=os.environ["API_KEY"])

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
)
print('\n LLM Response: ',response.text)


 LLM Response:  Since WWDC 2025 hasn't happened yet, I cannot provide any factual information about what Apple announced at that event. My knowledge is not based on future events.



#**LLM inference by augmneting the search results in prompt**

* In this part, we enhance the prompt by injecting real-time search results fetched via Serper into the input given to the LLM. The idea is to enrich the model’s context with up-to-date, factual information.

* As a result, the LLM is not just relying on its internal training data, but also leveraging current and relevant content retrieved from the web effectively simulating a lightweight RAG system.

* This method significantly improves the quality and accuracy of the response, especially for recent or dynamic queries like “What did Apple announce at WWDC 2025?”



In [ ]:
from google import genai

query = "What did Apple announce at WWDC 2025?"

search_results=search.run(query)
print('Search Result: ',search_results)

client = genai.Client(api_key=os.environ["API_KEY"])
prompt = f"Based on the following search results, answer the question: {query}\n\nSearch Results:\n{search_results}"

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt
)
print('\n LLM Response: ',response.text)

Search Result:  Apple's Biggest Reveals at WWDC 2025: iOS 26, Liquid Glass Design and Call Screening. The annual Apple developers conference delivered goodies for all devices, with fresh looks, updated operating systems and a whole basket of new capabilities.

 LLM Response:  Based on the search result, Apple announced the following at WWDC 2025:

*   **iOS 26**
*   **Liquid Glass Design**
*   **Call Screening**



In [ ]:
import google.generativeai as genai

# Correct way to configure
genai.configure(api_key=os.environ["API_KEY"])

# Load the Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-flash")  # or "gemini-1.5-pro"


In [ ]:
import google.generativeai as genai
import requests

# 1. Set up Gemini
# genai.configure(api_key="YOUR_GEMINI_API_KEY")
# model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# 2. Your query
query = "What did Apple announce at WWDC 2025?"

# 3. Use SerpAPI to search (not Serper)
params = {
    "engine": "google",
    "q": query,
    "api_key": os.environ["X-API-KEY"]
}
search_response = requests.get("https://serpapi.com/search", params=params)
search_results = search_response.json()
print("search_results", search_results)
# 4. Extract snippets from results
snippets = []
for result in search_results.get("organic_results", []):
    snippet = result.get("snippet")
    if snippet:
        snippets.append(snippet)

# 5. Build prompt for Gemini
summarized_results = "\n".join(snippets)
prompt = f"Based on the following search results, answer the question:\n{query}\n\nSearch Results:\n{summarized_results}"

# 6. Get response from Gemini
response = model.generate_content(prompt)
print("\nLLM Response (RAG):", response.text)


search_results {'search_metadata': {'id': '68406df6f357f690d6e38419', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/9adcc690bc02095d/68406df6f357f690d6e38419.json', 'created_at': '2025-06-04 16:01:58 UTC', 'processed_at': '2025-06-04 16:01:58 UTC', 'google_url': 'https://www.google.com/search?q=What+did+Apple+announce+at+WWDC+2025%3F&oq=What+did+Apple+announce+at+WWDC+2025%3F&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/9adcc690bc02095d/68406df6f357f690d6e38419.html', 'total_time_taken': 2.18}, 'search_parameters': {'engine': 'google', 'q': 'What did Apple announce at WWDC 2025?', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'What did Apple announce at WWDC 2025?', 'total_results': 11600000, 'time_taken_displayed': 0.32, 'organic_results_state': 'Results for exact spelling'}, 'inline_videos': [{'position': 1, 'title': 'What Apple Might Reveal at WWDC 2025?', 'link': 'https://www.youtube

#**OpenAI**

In [ ]:
import os
import requests
import json
from google import genai

# Set up API keys from environment variables
SERPER_API_KEY = os.environ.get("Open_SERPER_API_KEY")
api_key = os.environ.get("OpenAI_API_KEY")
organization_id = os.environ.get("OpenAI_Organization_ID")
project_id = os.environ.get("OpenAI_Project_ID")


In [ ]:
!pip install redis
!pip install openai==1.82.1
!pip install -U openai langchain-openai requests python-dotenv --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 4.6 MB/s eta 0:00:00


In [ ]:

import openai

# Create the OpenAI client with all required context
client = openai.OpenAI(
    api_key=api_key,
    organization=organization_id,
    project=project_id
)

# Make a chat completion request
response = client.chat.completions.create(
    model="gpt-4",  # Use a valid model name like "gpt-4" or "gpt-3.5-turbo"
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain quantum computing in simple terms."}
    ],
    temperature=0.7,
    max_tokens=200
)

# Print the assistant's reply
print(response.choices[0].message.content)


Quantum computing is a type of computing that uses quantum bits, or "qubits", to process information, instead of the regular bits used by traditional computers. 

In a normal computer, a bit is either a 0 or a 1. But a qubit, thanks to the principles of quantum mechanics, can be both a 0 and a 1 at the same time. This is called superposition. 

Additionally, there's another quantum property called entanglement, where qubits become interconnected and the state of one can instantly affect the state of another, no matter how far apart they are. 

These two properties make quantum computers incredibly powerful. They can process vast amounts of data and solve complex problems much faster than traditional computers. But they're also very delicate and hard to maintain, so we're still in the early stages of practical quantum computing.


#serp API

In [ ]:
import requests

params = {
    "engine": "google",
    "q": "Best vegan bakery in Brooklyn",
    "api_key": os.environ.get("X-API-KE"),
    "num": 3,  # Limit to 3 results
    "hl": "en",  # Language
    "gl": "us"   # Location
}

response = requests.get("https://serpapi.com/search", params=params)
results = response.json()

# Extracting light version of the organic results
light_results = []
for r in results.get("organic_results", [])[:3]:
    light_results.append({
        "title": r.get("title"),
        "snippet": r.get("snippet"),
        "link": r.get("link")
    })

print(light_results)



[{'title': 'Brooklyn Vegan Bakeries: 7 Spots for Cake, Donuts & More', 'snippet': 'The Best Vegan Bakeries to Visit in Brooklyn · Happy Zoe Vegan Bakery · Clementine Café & Bakery · Dun-Well Doughnuts · Cloudy Donut · Reverie.', 'link': 'https://veggiesabroad.com/brooklyn-vegan-bakery-guide/'}, {'title': 'BEST of Brooklyn, NY Vegan Bakery', 'snippet': 'Top 10 Best Vegan Bakery Near Brooklyn, New York · 1. Happy Zoe Vegan Bakery · 2. Vegan Pastry Lab · 3. Clementine Bakery Cafe · 4. Ladybird Bakery · 5. Le ...', 'link': 'https://www.yelp.com/search?find_desc=Vegan+Bakery&find_loc=Brooklyn%2C+NY'}]


# GPT+SERP API

In [ ]:
# ✅ Step 1: Install dependencies
# !pip install requests python-dotenv --quiet
# !pip install --upgrade --no-cache-dir openai

# ✅ Step 2: Import libraries
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv

# ✅ Step 3: Set API keys
api_key = os.environ.get("OpenAI_API_KEY")
organization_id = os.environ.get("OpenAI_Organization_ID")
project_id = os.environ.get("OpenAI_Project_ID")
SERPER_API_KEY = os.environ.get("Open_SERPER_API_KEY")

# ✅ Step 4: Create OpenAI client
client = OpenAI(
    api_key=api_key,
    organization=organization_id,
    project=project_id
)

# ✅ Step 5: Define Serper function
def get_light_search_results(query, num_results=3):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPER_API_KEY,
        "num": num_results
    }
    response = requests.get(url, params=params)
    results = response.json().get("organic_results", [])
    return [
        {
            "title": r.get("title"),
            "snippet": r.get("snippet"),
            "link": r.get("link")
        } for r in results if r.get("title") and r.get("snippet")
    ]

# ✅ Step 6: Build prompt from results
def build_prompt(query, serp_results):
    base_prompt = f"Generate JSON microsite content from the following search results.\nQuery: {query}\n\n"
    for r in serp_results:
        base_prompt += f"- {r['title']}: {r['snippet']}\n"
    base_prompt += (
        "\nReturn strictly JSON like this:\n"
        "{ \"title\": \"...\", \"subtitle\": \"...\", \"bullets\": [\"...\", \"...\", \"...\"] }"
    )
    return base_prompt

# ✅ Step 7: Call GPT-4o
def generate_microsite_data(query):
    search_results = get_light_search_results(query)
    prompt = build_prompt(query, search_results)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a JSON microsite generator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5,
        max_tokens=500
    )
    return response.choices[0].message.content

# ✅ Step 8: Run it!
query = "Best vegan bakeries in Brooklyn"
result = generate_microsite_data(query)
print("result",result)


result ```json
{
  "title": "Best Vegan Bakeries in Brooklyn",
  "subtitle": "Discover the Top Spots for Vegan Treats",
  "bullets": [
    "Happy Zoe Vegan Bakery",
    "Clementine Café & Bakery",
    "Dun-Well Doughnuts",
    "Cloudy Donut",
    "Reverie",
    "Vegan Pastry Lab",
    "Ladybird Bakery"
  ]
}
```
